# Strava scraper

This is the first attempt at scraping all cycling routes posted on strava located in **Spain**. We only need to download the *gpx* files.

In [2]:
#Importing libraries.

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [5]:
#Initializing our webdriver (Chrome).

driver = webdriver.Chrome()

In [44]:
#Entering a ride.

driver.get('https://www.strava.com/activities/5798394213')

In [8]:
#Clicking the three-dot menu.

menu = driver.find_element_by_xpath('/html/body/div[2]/div[3]/nav/div/div')
menu.click()

In [9]:
#Downloading the gpx.

gpx = driver.find_element_by_xpath('/html/body/div[2]/div[3]/nav/div/div/ul/li[3]')
gpx.click()

In [39]:
#Obtaining the ride type. This will only match "Bicicleta" if it's a bike ride.

ridetype = driver.find_element_by_xpath('//*[@id="heading"]/header/h2/span').text[-9:]
ridetype

'Bicicleta'

In [28]:
#Obtaining the location:

location = driver.find_element_by_xpath('//*[@id="heading"]/div/div/div[1]/div/div/span').text
location

'Londres, Reino Unido'

In [ ]:
#Naming schema for download links.

'http://strava.com/activities/5798394100/export_gpx'

Names of spanish CCAAs (or other territories) as they are used on **Strava**:

España
Asturias
Cataluña
Madrid
Aragón
La Rioja


In [84]:
#Making a list of the names we want to detect:

names = ['España', 'Asturias', 'Cataluña', 'Madrid', 'Aragón', 'La Rioja', 'Andalucía', 'Comunidad Valenciana', 'Andorra']

In [82]:
start = time.time()

for i in range(10):
    try:
        base_url = 'https://www.strava.com/activities/'
        base_number = 3981642097
        url = base_url + str((base_number + i))
        time.sleep(0.1)
        driver.get(url)
        time.sleep(0.1)
        if driver.find_element_by_xpath('//*[@id="heading"]/header/h2/span').text[-9:] == 'Bicicleta':
            for i in names:
                if i in driver.find_element_by_xpath('//*[@id="heading"]/div/div/div[1]/div/div/span').text:
                    gpx_url = url + '/export_gpx'
                    driver.get(gpx_url)
                    time.sleep(0.1)
                else:
                    pass
        else:
            pass
    except:
        pass
    
stop = time.time() 
duration = (stop - start) / 60
print('Minutes:', duration)

Minutes: 8.989719223976135
